# Regularisation by Kamishima - Adult data

This notebook contains the implementation of the in-processing fairness intervention introduced in [Fairness-Aware Classifier with Prejudice Remover Regularizer](https://link.springer.com/chapter/10.1007/978-3-642-33486-3_3) by Kamishima et al. (2012) as part of the IBM AIF360 fairness tool box github.com/IBM/AIF360.

The intervention achieves demographic parity in a logistic regression classifier which is based on maximising the sum between utility expressed via probabilities of classifying data points correctly given their features and further a regularisation term that incorporates the level of unfairness in the classifier.

In [ ]:
from pathlib import Path

import joblib
import pandas as pd
import numpy as np
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import PrejudiceRemover  # noqa
from fairlearn.metrics import demographic_parity_difference
from helpers.metrics import accuracy
from helpers.plot import group_box_plots

In [ ]:
from helpers import export_plot

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out hte preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "adult"

train = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

In order to process data for our fairness intervention we need to define special dataset objects which are part of every intervention pipeline within the IBM AIF360 toolbox. These objects contain the original data as well as some useful further information, e.g., which feature is the protected attribute as well as which column corresponds to the label.

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)

## Train unfair model

For maximum reproducibility we load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [ ]:
bl_model = joblib.load(artifacts_dir / "models" / "finance" / "baseline.pkl")

Get predictions for the validation and test data

In [ ]:
bl_test_probs = bl_model.predict_proba(test.drop("salary", axis=1))[:, 1]
bl_test_pred = bl_test_probs > 0.5
test_sds_pred = test_sds.copy(deepcopy=True)

## Demographic parity

Here, we address demographic parity by learning a fair logisitc regression on the trainig data. Subsequently, we apply the learnt model to the test data and analyse the outcomes for fairness and accuracy. The model training allows the specification of a parameter $\eta$, which controls the loss in accuracy for an increase in fairness. The larger eta the higher the obtained fairness on average. 

The user is encouraged to consider the influence of the choice of the parameter $\eta$. However, since the learning of the fair model is for some choice of $\eta$ quite slow, we generated the following predictions with $\eta=10$. Since we cannot save the model we saved the predictions generated by the model instead. Since the training is deterministic, the same choice of $\eta$ should lead to same results.

In [ ]:
# PR = PrejudiceRemover(eta=10.0, sensitive_attr="sex", class_attr="salary")
# PR.fit(train_sds)

Apply model on test data.

In [ ]:
# test_sds_pred = PR.predict(test_sds)
# test_probs = test_sds_pred.scores.flatten()

In [ ]:
test_probs = np.load(
    artifacts_dir / "models" / "finance" / "kamishima_test_scores.npy"
)
test_pred = test_probs > 0.5

Analyse fairness and accuracy

In [ ]:
test_features = test.drop(columns=["sex", "salary"]).values
test_sex = test.sex.values
test_salary = test.salary.values
mask = test_sex == 1

# baseline metrics
bl_test_acc = accuracy(test_salary, bl_test_probs)
bl_test_dpd = demographic_parity_difference(
    test.salary, bl_test_pred, sensitive_features=test_sex,
)

# new model metrics
test_acc = accuracy(test_salary, test_pred)
test_dpd = demographic_parity_difference(
    test.salary, test_pred, sensitive_features=test_sex,
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline demographic parity: {bl_test_dpd:.3f}")
print(f"Demographic parity: {test_dpd:.3f}\n")

In [ ]:
dp_box = group_box_plots(
    np.concatenate([bl_test_probs, test_probs]),
    np.tile(test.sex.map({0: "Female", 1: "Male"}), 2),
    groups=np.concatenate(
        [np.zeros_like(bl_test_probs), np.ones_like(test_probs)]
    ),
    group_names=["Baseline", "Kamishima"],
    title="Distribution of scores by sex",
    xlabel="Scores",
    ylabel="Method",
)
dp_box

In [ ]:
export_plot(dp_box, "kamishima-dp.json")